Predict a chess player's FIDE Elo rating from one game
=============================
This is my attempt at the Kaggle Competition [Finding Elo](http://www.kaggle.com/c/finding-elo). As the title suggests, we are given records of 50000 chess games. 25000 of those records have the FIDE Elo ratings of the two players each. And the other 25000 records are missing the Elo ratings of the players. Those Elo ratings are for us to predict based on the moves played (given in the game records) and the 25000 training records.

For our convenience Kaggle also passed all the games through a very powerful chess engine(stockfish) which analyses the board after each move and measures the strength of the board (i.e. how much the board is in white's favor). We have that data available in a separate csv file.

The Data
-------
This is what the game records look like. (File: `data.pgn`)

```
[Event "1"] <---------------------Event ids go from 1 to 50000
[Site "kaggle.com"] <-------------Useless fields
[Date "??"]<--------------|
[Round "??"]<-------------|
[White "??"]<-------------|
[Black "??"]<-------------|
[Result "1/2-1/2"]<----------------Results can be 1-0 0-1 or 1/2-1/2(draw)
[WhiteElo "2354"]<-----------------First 25000 games have Elo ratings of the players,
[BlackElo "2411"]<-------------|       The other 25000 games are missing this data ("??")

1. Nf3 Nf6 2. c4 c5 3. b3 g6 4. Bb2 Bg7 5. e3 O-O 6. Be2 b6 7. O-O Bb7 8.
Nc3 Nc6 9. Qc2 Rc8 10. Rac1 d5 11. Nxd5 Nxd5 12. Bxg7 Nf4 13. exf4 Kxg7 14.
Qc3+ Kg8 15. Rcd1 Qd6 16. d4 cxd4 17. Nxd4 Qxf4 18. Bf3 Qf6 19. Nb5 Qxc3
1/2-1/2           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                  This is the series of moves ending with the result
[Event "2"]
[Site "kaggle.com"]
...
49999 more records follow
```

Auxilliary Data (very useful!)
----------------------------
The `stockfish.csv` file looks as follows.
```
1,18 17 12 8 -5 12 3 -2 22 21 20 13 8 21 11 3 -6 5 1 -10 -21 -1 -26 18 48 48 53 73 46 68 51 60 54 46 70 62 35 54
2,26 44 26 18 14 34 36 31 37 35 42 52 55
...
50000 such lines... one per game
```
It's a two column csv where first column is the index of the game and the second is a space-seperated series of  numbers, the numbers denote the board strength after each move.

### Disclaimers
- This is my first Kaggle Competition
- Most of this code was written after the competition ended
- I didn't consult any of the published solutions to write this code(although consulted stackoverflow heavily for numpy recipes). I might study some of the solution and write a new solution and publish separately.

The Solution
============

Data extraction before Python
---------------------------
I used shell scripts to make two files `whiteelo.dat` and `blackelo.dat` which contained the ordered list of 25000 elo rankings each.
```
2354
2523
1915
...
25000 such lines
```

Similarly I generated a `results.dat` which contained the results of the 50000 games. `1` means White won, `0` means White lost, `1/2` means it was a draw.
```
1/2
1/2
1/2
1/2
0
0
1
1
1
...
50000 such lines
```

### Initialize Python

In [1]:
import csv
import scipy
from scipy import stats
import warnings

warnings.simplefilter("error") #For better debugging and clean output, let any warning throw an exception

Parsing Simple files
-------------------
The `whiteelo.dat` `blackelo.dat` and `result.dat` files that I generated earlier(using shell/perl commandline) are easy to load into python as arrays. So we start with that.

`whiteelo.dat` and `blackelo.dat` have the list of known Elo rankings and `result.dat` have results of all the 50000 games.

In [2]:
whiteEloFile = csv.reader(open('data/whiteelo.dat', 'r'))
blackEloFile = csv.reader(open('data/blackelo.dat', 'r'))

whiteElo=[int(x[0]) for x in whiteEloFile]
blackElo=[int(x[0]) for x in blackEloFile]


def parseResult(res):
    if(res=="1/2"):
        return 0.5
    else:
        return int(res)

matchResultsFile = csv.reader(open('data/results.dat', 'r'))
matchResults = [parseResult(x[0]) for x in matchResultsFile]

Feature Extraction
-----------------
This is the meat of the code. All we have is a series of move from the original data(`data.pgn`) and the stockfish chess engine board analysis after each move in each game.

For now we will ignore the series of moves and focus on `stockfish.csv` since Kaggle already provided the quality of each move through the stockfish analysis.

### Converting Board Scores to Features
Points to note with board scores are...
- We are trying to predict two Elo scores, white player's and black player's. But we have one series of board strengths. For each move by a white player(or black player), the quality of that move is signified by how much that score improves the board score. One way to compute that is the difference between the score before that particular move and after. So we should convert the score series into a series of differences between consecutive scores.
- The two consecutive numbers in this new series of score differences, two consecutive numbers will denote performance of opposite players, thus the move quality of a white player's moves and black players moves are interleaved and should be separated.
- The length of move series could be indefinitely long or extremely short (even zero moves in case of forfeiture)
- Because the length of the series is variable(and for other intuitive reasons) making each move's score a separate feature won't work. What we need to do is figure out the overall performance of a player based on the performance of all his moves(we can also use the performance of the opposite player as a hint to what he was up against)
- One way to transform the series of move performances into overall performance is to do statistical analysis on the series. `scipy.stats.describe` provides a convenient way to transform the series into a set of features which all give useful info about the player's performance. The values returned are number-of-moves, min, max, mean, variance, skewness and kurtosis. Now we have a fixed number of features for all the games. Some of these features are obviously interesting...
  - mean: this will tell us the average performance of the player which obviously can tell us how well he can play.
  - variance: this will point to the consistency of player. Is he all over the place or is consistently performing better
  - skewness: this feature should tell us whether a few good moves by fluke are tipping the scale for any player.
- In case of forfeiture, we really don't have any data to go by, and I think our best bet is to just predict the average score and hope for the best, further, the training data from forfeiture games could mess with prediction of other games, so we should take out those records from the training data and treat forfeitures completely separately.
- In case of draws we can use the difference in number of moves between white player and black player(or total_moves%2) to decide who offered the draw, which could be significant.

### Further Refinements to the Feature Matrix and the Model
I've implemented the following refinements...
- Added some 2nd degree features by multiplying the number of observation with the other features
- Split the model into three outcome based datasets (white wins, black wins, draw)
These two improvements(and a bug fix) combined gave a slight boost to the performance jump of around 22 percentile.

### Notes on Parsing stockfish.csv series
For the most part stockfish data is just a series of space separated integers, so very easy to parse. But we have to take care of a couple of edge cases.
- In forfeiture games where only white player makes one move and the black player doesn't show up or when white player doesn't show up, we get an empty series, which needs to be explicitly converted to a series of just one move.
- Another edge case is, when the game goes into check-mate or when there's only one legal move(a king trying to run and hide) the file has `NA` instead of an integer, in which case the previous board strength number is good enough(IMO)

Here is a function to parse the stockfish scores, taking care of the edge cases.

In [3]:
def chessScore(x):
    if x == '':
        return 0
    if x != 'NA':
        chessScore.previous = int(x)
    return chessScore.previous

And here is the code that will generate the feature matrix.

First we make the matrix out of statistical features returned by `scipy.stats.describe` for both white player and black player separately, and difference in number of moves between the two players(which will be 0 or one).

In [4]:
moveStatsList=[]
index = 0
with open('data/stockfish.csv', 'r') as stockfishScoresFile:
    stockfishScoreSheet = csv.reader(stockfishScoresFile)
    
    next(stockfishScoreSheet) #skip the header line
    forfeitcount = 0
    for x in stockfishScoreSheet:
        boardScores = scipy.array([chessScore(x) for x in x[1].split(" ")])
        initScores = scipy.array([0]);
        initScores = scipy.append(initScores, boardScores[:-1])
        moveScores = boardScores - initScores
        whiteScores = moveScores[::2]
        blackScores = moveScores[1::2]
        if(type(x) is not list):
            print ("x is not list: ", x)
        index += 1
        if(blackScores.size <= 1):
            forfeitcount += 1
            #Ignore forfeiture/extremely short game for training.
            #print ("In a forfeiture game :", forfeitcount, " :", index)
            (wnobs, (wmin, wmax), wmean, wvariance, wskewness, wkurtosis) \
                = (0,(0,0),0,0,0,0);
            (bnobs, (bmin, bmax), bmean, bvariance, bskewness, bkurtosis) \
                = (0,(0,0),0,0,0,0);
        else:
            (wnobs, (wmin, wmax), wmean, wvariance, wskewness, wkurtosis) \
                = scipy.stats.describe(whiteScores)
            (bnobs, (bmin, bmax), bmean, bvariance, bskewness, bkurtosis) \
                = scipy.stats.describe(blackScores)
        moveStatsList.append(scipy.array(
                [wnobs, wmin, wmax, wmean, wvariance, wskewness, wkurtosis,
                 wnobs*wmin, wnobs*wmax, wnobs*wmean, wnobs*wvariance, wnobs*wskewness, wnobs*wkurtosis,
                 bnobs, bmin, bmax, bmean, bvariance, bskewness, bkurtosis,
                 bnobs*bmin, bnobs*bmax, bnobs*bmean, bnobs*bvariance, bnobs*bskewness, bnobs*bkurtosis,
                 bnobs - wnobs]))


#### Add results to the feature matrix for each game

In [5]:
moveStats = scipy.array(moveStatsList)
moveStats = scipy.column_stack((moveStats, matchResults))
print("Feature matrix shape is: ", moveStats.shape)

Feature matrix shape is:  (50000, 28)


#####Add an index to the feature list so we keep track of individual rows

In [6]:

moveStats = scipy.insert(moveStats, 0, values=range(1, 50001), axis=1)

Future Directions for Features
----------------------------
- Long hard fought matches should be more significant to guaging performance of the players so multiplying features like variance and mean with the number of moves could provide an interesting feature.(DONE)
- We could in general multiply any two features and create a bunch of 2nd degree polynomials without worrying about overfitting(25000 observations) and see what sticks.
- Sepearating Draws and making separate models for them could make a better predictor for the games with results.(DONE)

##Final Steps
The following code does the final modeling and prediction from the feature data. Here are the steps taken...
- Separate training and prediction rows
- Combine the Training Elo Ratings with the training data.
- Take out rows for extremely short games (mostly forfeitures I presume) and deal with them separately, and simplistically.
- Create two seperate models, one for white player and one for black player. Both will use the same feature sets.
- Make predictions for the other 25000 games
- Use average Elo ratings for the extremely short games that we ignored
- combine the indices of the games and the results and dump them to a csv file.

In [8]:
# separate training and prediction rows
trainingStats = moveStats[0:25000,:]
predictionStats = moveStats[25000:, :]
# couple the known elo rankings to the training set so we keep track of it wrt indices
trainingStats = scipy.insert(trainingStats, 1, values=whiteElo, axis=1)
trainingStats = scipy.insert(trainingStats, 2, values=blackElo, axis=1)
# further split both into regular and forfeit data
# From: http://stackoverflow.com/questions/11453141
trainingStatsReg = trainingStats[trainingStats[:,3]!=0]
trainingStatsForf = trainingStats[trainingStats[:,3]==0]
predictionStatsReg = predictionStats[predictionStats[:,3]!=0]
predictionStatsForf = predictionStats[predictionStats[:,3]==0]

# Make separate models for win/lose/draws
trainingStatsDraws = trainingStatsReg[trainingStatsReg[:,-1]==0.5]
trainingStatsWins = trainingStatsReg[trainingStatsReg[:,-1]==1]
trainingStatsLosses = trainingStatsReg[trainingStatsReg[:,-1]==0]
predictionStatsDraws = predictionStatsReg[predictionStatsReg[:,-1]==0.5]
predictionStatsWins = predictionStatsReg[predictionStatsReg[:,-1]==1]
predictionStatsLosses = predictionStatsReg[predictionStatsReg[:,-1]==0]
# Run the regular data through two linear models, one for white and one for black

#from http://stackoverflow.com/questions/11479064/multivariate-linear-regression-in-python
from sklearn import linear_model
whiteModelWins = linear_model.LinearRegression()
whiteModelWins.fit(trainingStatsWins[:,3:],trainingStatsWins[:,1])
blackModelWins = linear_model.LinearRegression()
blackModelWins.fit(trainingStatsWins[:,3:],trainingStatsWins[:,2])

whiteModelLosses = linear_model.LinearRegression()
whiteModelLosses.fit(trainingStatsLosses[:,3:],trainingStatsLosses[:,1])
blackModelLosses = linear_model.LinearRegression()
blackModelLosses.fit(trainingStatsLosses[:,3:],trainingStatsLosses[:,2])

whiteModelDraws = linear_model.LinearRegression()
whiteModelDraws.fit(trainingStatsDraws[:,3:],trainingStatsDraws[:,1])
blackModelDraws = linear_model.LinearRegression()
blackModelDraws.fit(trainingStatsDraws[:,3:],trainingStatsDraws[:,2])


whitePredictionsWins = whiteModelWins.predict(predictionStatsWins[:,1:])
blackPredictionsWins = blackModelWins.predict(predictionStatsWins[:,1:])

whitePredictionsLosses = whiteModelLosses.predict(predictionStatsLosses[:,1:])
blackPredictionsLosses = blackModelLosses.predict(predictionStatsLosses[:,1:])

whitePredictionsDraws = whiteModelDraws.predict(predictionStatsDraws[:,1:])
blackPredictionsDraws = blackModelDraws.predict(predictionStatsDraws[:,1:])

# for forfeiture make averages of white elo and black elo for white forfeiture and
# black forfeiture each
whitePredictionsForf = scipy.mean(trainingStatsForf[:,1])*scipy.ones(predictionStatsForf.shape[0])
blackPredictionsForf = scipy.mean(trainingStatsForf[:,2])*scipy.ones(predictionStatsForf.shape[0])

#stitch the output together
# Stitch index and predictions
predictionsWins = scipy.column_stack((predictionStatsWins[:,0],whitePredictionsWins,blackPredictionsWins))
predictionsLosses = scipy.column_stack((predictionStatsLosses[:,0],whitePredictionsLosses,blackPredictionsLosses))
predictionsDraws = scipy.column_stack((predictionStatsDraws[:,0],whitePredictionsDraws,blackPredictionsDraws))
predictionsForf = scipy.column_stack((predictionStatsForf[:,0],whitePredictionsForf,blackPredictionsForf))

predictions = scipy.concatenate((predictionsWins, predictionsLosses, predictionsDraws, predictionsForf))
predictions = scipy.around(predictions)
predictions = predictions.astype(int)
predictions = predictions[predictions[:,0].argsort()]
#spit out csv
scipy.savetxt("results.csv", predictions, fmt="%d", delimiter=",", header="Event,WhiteElo,BlackElo", comments='')

Here's a sample of our results.

In [9]:
print(predictions[0:10])

[[25001  2317  2333]
 [25002  2074  2208]
 [25003  2320  2231]
 [25004  2334  2349]
 [25005  2272  2305]
 [25006  2091  2281]
 [25007  2276  2114]
 [25008  2350  2368]
 [25009  2374  2230]
 [25010  2310  2159]]
